In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['FMS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('fibromyalgia syndrome', 66),
 ('fundamental movement skills', 8),
 ('functional mesoporous silica', 6),
 ('functional magnetic stimulation', 5),
 ('fluorous mixture synthesis', 4),
 ('fundamental motor skills', 4),
 ('fimasartan', 3)]

In [ ]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-17 01:53:47] indra.ontology.bio.ontology - Initializing INDRA bio ontology for the first time, this may take a few minutes...
INFO: [2020-10-17 01:53:47] indra.ontology.bio.ontology - Adding nodes...
INFO: [2020-10-17 01:53:56] indra.ontology.bio.ontology - Adding xrefs...
INFO: [2020-10-17 01:54:08] indra.ontology.bio.ontology - Adding hierarchy...
INFO: [2020-10-17 01:54:25] indra.ontology.bio.ontology - Building name lookup...
INFO: [2020-10-17 01:54:31] indra.ontology.bio.ontology - Finished initializing bio ontology...
INFO: [2020-10-17 01:54:31] indra.ontology.bio.ontology - Caching INDRA bio ontology at /home/ubuntu/.indra/bio_ontology/1.3/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'real time cell analyser': 'ungrounded',
  'real time cell analysis': 'ungrounded',
  'real time cell analyzer': 'ungrounded'},
 {},
 []]

In [5]:
grounding_map, names, pos_labels = ({'fibromyalgia': 'MESH:D005356',
  'fibromyalgia syndrome': 'MESH:D005356',
  'fimasartan': 'CHEBI:CHEBI:136044',
  'fluorous mixture synthesis': 'ungrounded',
  'functional magnetic stimulation': 'MESH:D055909',
  'functional mesoporous silica': 'ungrounded',
  'fundamental motor skills': 'ungrounded',
  'fundamental movement skills': 'ungrounded'},
 {'MESH:D005356': 'Fibromyalgia',
  'CHEBI:CHEBI:136044': 'fimasartan',
  'MESH:D055909': 'Magnetic Field Therapy'},
 ['CHEBI:CHEBI:136044', 'MESH:D005356', 'MESH:D055909'])

In [6]:
excluded_longforms = []

In [7]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [8]:
additional_entities = {'HGNC:2433': ['CSF1R', ['C-FMS', 'FMS']]}

In [9]:
unambiguous_agent_texts = {}

In [10]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [11]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [12]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [13]:
intersection1

[('HGNC:2433', 'HGNC:2433', 806)]

In [14]:
intersection2

[('MESH:D005356', 'HGNC:2433', 0),
 ('MESH:D055909', 'HGNC:2433', 0),
 ('ungrounded', 'HGNC:2433', 0),
 ('CHEBI:CHEBI:136044', 'HGNC:2433', 0)]

In [15]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [16]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 04:37:57] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 04:38:00] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9390988190575305 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'MESH:D005356': 47,
  'MESH:D055909': 5,
  'ungrounded': 17,
  'CHEBI:CHEBI:136044': 2,
  'HGNC:2433': 74},
 'f1': {'mean': 0.939099, 'std': 0.051149},
 'precision': {'mean': 0.927636, 'std': 0.063949},
 'recall': {'mean': 0.953231, 'std': 0.038203},
 'HGNC:2433': {'f1': {'mean': 0.980603, 'std': 0.025354},
  'pr': {'mean': 0.986667, 'std': 0.026667},
  'rc': {'mean': 0.976471, 'std': 0.047059}},
 'MESH:D055909': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'ungrounded': {'f1': {'mean': 0.937778, 'std': 0.081225},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 0.96, 'std': 0.08}},
 'MESH:D005356': {'f1': {'mean': 0.94907, 'std': 0.045631},
  'pr': {'mean': 0.977778, 'std': 0.044444},
  'rc': {'mean': 0.924444, 'std': 0.065697}},
 'CHEBI:CHEBI:136044': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}}}

In [22]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [23]:
disamb.dump(model_name, results_path)

In [24]:
print(disamb.info())

Disambiguation model for FMS

Produces the disambiguations:
	CSF1R*	HGNC:2433
	Fibromyalgia*	MESH:D005356
	Magnetic Field Therapy*	MESH:D055909
	fimasartan*	CHEBI:CHEBI:136044

Class level metrics:
--------------------
Grounding             	Count	F1     
                 CSF1R*	74	 0.9806
          Fibromyalgia*	47	0.94907
            Ungrounded	17	0.93778
Magnetic Field Therapy*	 5	    0.6
            fimasartan*	 2	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.9391
	Precision:	0.92764
	Recall:		0.95323

* Positive labels
See Docstring for explanation



In [25]:
model_to_s3(disamb)